OPIXray是用于X光违禁品检测的数据集，此部分代码用于将OPIXray数据集中，所有的违禁品提取出，并保存为图片文件，用于进行图像生成和图像分类。

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# 输入危险品名称，返回其在数组的序号
def get_class_index(my_class):
    classes = {'Folding_Knife' : 0, 'Multi-tool_Knife' : 1, 'Scissor' : 2, 'Straight_Knife' : 3, 'Utility_Knife' : 4}
    return classes.get(my_class)

if not os.path.exists('/dataset/OPIXray_Crop_DM/'):
    os.mkdir('/dataset/OPIXray_Crop_DM/')
if not os.path.exists('/dataset/OPIXray_Crop_DM/train/'):
    os.mkdir('/dataset/OPIXray_Crop_DM/train/')
if not os.path.exists('/dataset/OPIXray_Crop_DM/test/'):
    os.mkdir('/dataset/OPIXray_Crop_DM/test/')
if not os.path.exists('/dataset/OPIXray_Crop_DM/val/'):
    os.mkdir('/dataset/OPIXray_Crop_DM/val/')
if not os.path.exists('/dataset/OPIXray_Crop_DM/meta/'):
    os.mkdir('/dataset/OPIXray_Crop_DM/meta/')

    
train_file = open('/dataset/OPIXray_Crop_DM/meta/train.txt', 'a')
with open('/dataset/OPIXray/train/train_knife.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = line.strip('\n')
        img = Image.open('/dataset/OPIXray/train/train_image/' + line + '.jpg')
        
        with open('/dataset/OPIXray/train/train_annotation/' + line + '.txt', 'r') as f:
            annotation = f.readline()
            annotation = annotation.strip('\n')
            results = annotation.split(' ')
            # if not os.path.exists('/dataset/OPIXray/train/train_crop/' + results[1]):
                # os.mkdir('/dataset/OPIXray/train/train_crop/' + results[1])
            img_crop = img.crop((int(results[2]), int(results[3]), int(results[4]), int(results[5])))
            # img_crop.save('/dataset/OPIXray/train/train_crop/' + results[1] + '/' + line + '_crop.jpg')
            file_path = '/dataset/OPIXray_Crop_DM/train/' + line + '_crop.jpg'
            img_crop.save(file_path)
            
            # train_file.write(file_path + ' ' + str(get_class_index(results[1])) + '\n')
            train_file.write(line + '_crop.jpg' + ' ' + str(get_class_index(results[1])) + '\n')

train_file.close()

test_file = open('/dataset/OPIXray_Crop_DM/meta/test.txt', 'a')
with open('/dataset/OPIXray/test/test_knife.txt', 'r') as f:
    data = f.readlines()
    for line in data:
        line = line.strip('\n')
        img = Image.open('/dataset/OPIXray/test/test_image/' + line + '.jpg')
        
        with open('/dataset/OPIXray/test/test_annotation/' + line + '.txt', 'r') as f:
            annotation = f.readline()
            annotation = annotation.strip('\n')
            results = annotation.split(' ')
         # if not os.path.exists('/dataset/OPIXray/test/test_crop/' + results[1]):
                # os.mkdir('/dataset/OPIXray/test/test_crop/' + results[1])
            img_crop = img.crop((int(results[2]), int(results[3]), int(results[4]), int(results[5])))
            # img_crop.save('/dataset/OPIXray/test/test_crop/' + results[1] + '/' + line + '_crop.jpg')
            file_path = '/dataset/OPIXray_Crop_DM/test/' + line + '_crop.jpg'
            img_crop.save(file_path)
            
            # test_file.write(file_path + ' ' + str(get_class_index(results[1])) + '\n')
            test_file.write(line + '_crop.jpg' + ' ' + str(get_class_index(results[1])) + '\n')
            
test_file.close()

In [ ]:
# 由于源数据集OPIXray中，没有val，因此，需要重新划分。数据量总数为8885，按照train:test:val=8:1:1的比例划分，分别为：7109:888:888，随机划分。
import os

import random

# 读取文件1
with open("/dataset/OPIXray_Crop_DM/meta/train.txt", "r") as file1:
    data1 = file1.readlines()

# 读取文件2
with open("/dataset/OPIXray_Crop_DM/meta/test.txt", "r") as file2:
    data2 = file2.readlines()

# 合并两个文件的数据
data = data1 + data2

# 打乱数据的顺序
random.shuffle(data)

# 将数据分成8:1:1的比例
split_index1 = int(len(data) * 0.8)
split_index2 = int(len(data) * 0.9)

# 将数据写入三个文件
with open("/dataset/OPIXray_Crop_DM/meta/file_train.txt", "w") as file_train:
    file_train.writelines(data[:split_index1])

with open("/dataset/OPIXray_Crop_DM/meta/file_val.txt", "w") as file_val:
    file_val.writelines(data[split_index1:split_index2])

with open("/dataset/OPIXray_Crop_DM/meta/file_test.txt", "w") as file_test:
    file_test.writelines(data[split_index2:])
    
# 上述执行完后，需要手动修改三个txt文件的文件名，并删除原来的两个文件

In [ ]:
# 将图片文件移动到对应的文件夹内
import os,shutil

# 训练图片文件名
train_filenames = []
with open("/dataset/OPIXray_Crop_DM/meta/train.txt", "r") as file_train:
    for line in file_train:
        train_filenames.append(line.strip())

# 验证图片文件名
val_filenames = []
with open("/dataset/OPIXray_Crop_DM/meta/val.txt", "r") as file_val:
    for line in file_val:
        val_filenames.append(line.strip())

# 测试图片文件名
test_filenames = []
with open("/dataset/OPIXray_Crop_DM/meta/test.txt", "r") as file_test:
    for line in file_test:
        test_filenames.append(line.strip())

# 移动文件的位置
# shutil.move(origin_file, dst_folder)

# 源文件夹路径
src_folder = "/dataset/OPIXray_Crop_DM/"
# 目标文件夹路径
dst_folder = "/dataset/OPIXray_Crop_DM/"

# 图片文件名
train_file_path = "/dataset/OPIXray_Crop_DM/train/"
test_file_path = "/dataset/OPIXray_Crop_DM/test/"
files = os.listdir(train_file_path)
files.append(os.listdir(test_file_path))
for filename in files:
    for train_file in train_filenames:
        if filename == train_file.split(' ')[0]):
            shutil.move("/dataset/OPIXray_Crop_DM/train/"+filename, )
        
        
# for file in files:
    # print(file)
    # src_path = os.path.join(src_folder, file.split(' ')[0])